In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain_pinecone import PineconeVectorStore
from ctransformers import AutoModelForCausalLM
from langchain.base_language import BaseLanguageModel
from langchain_community.llms import CTransformers

c:\Users\DELL\anaconda3\envs\mchatbot-v2\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
PINECONE_API_KEY = "pcsk_7V4oZT_CZErCprEGzsgWYxCUtCMaVwNmFmvKcST7RxqXoUnTvvWHoo9gMBxxHMaeNWrTBH"
PINECONE_API_ENV = "us-east-1"

In [3]:
#Extract data from PDFs
def load_pdf_data(data):
    pdf_loader = DirectoryLoader(data,
                                 glob="*.pdf",
                                 loader_cls=PyPDFLoader)
    documents = pdf_loader.load()

    return documents

In [4]:
extracted_data = load_pdf_data("data/")

In [5]:
# extracted_data

In [6]:
#Split the text into smaller chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    

    return text_chunks

In [7]:
text_chunks = text_split(extracted_data)
print(len(text_chunks))

7020


In [8]:
#download embedding model
def download_embedding_model():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_embedding_model()

C:\Users\DELL\AppData\Local\Temp\ipykernel_9352\3594143886.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [10]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [11]:
query_result =  embeddings.embed_query("Hello, how are you?")
print(len(query_result))

384


In [12]:
# import os

# os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

# index_name = "medicalchatbot-v2"

# vectorstore_from_texts = PineconeVectorStore.from_texts(
#     [t.page_content for t in text_chunks],
#     index_name=index_name,
#     embedding=embeddings
#     )

In [13]:
#If we already have an index we can load it like this
import os

os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

index_name = "medicalchatbot-v2"
docsearch = PineconeVectorStore.from_existing_index(index_name, embeddings)
query = "What are Allergies"
doc = docsearch.similarity_search(query, k=3)
print(doc)


[Document(id='d54ecb7c-bae6-40bf-aafd-5a38dcd0ab4f', metadata={}, page_content="GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies\nAllergic rhinitis is commonly triggered by\nexposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.\nThe presence of an allergen causes the\nbody's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.\nIgE molecules attach to mast\ncells, which contain histamine.HistaminePollen grains\nLymphocyte\nFIRST EXPOSURE"), Document(id='3d45d006-fd59-457f-a08b-bd6b221e77b9', metadata={}, page_content='allergens are the following:\n• plant pollens\n• animal fur and dander\n• body parts from house mites (microscopic creatures\nfound in all houses)\n• house dust• mold spores• cigarette smoke• solvents• cleaners\nCommon food allergens include the following:\n• nuts, especially peanuts, walnuts, and brazil nuts\n• fish, mollusks, and

In [14]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [15]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [16]:
config = {'max_new_tokens':512, 'temperature':0.8}
llm = CTransformers(model='model/llama-2-7b-chat.ggmlv3.q4_0.bin', model_type='llama', lib='avx', config=config)

In [17]:
qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

C:\Users\DELL\AppData\Local\Temp\ipykernel_9352\4216317267.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result=qa({"query": user_input})
